# Финальное задание по анализу данных из Dota2

In [3]:
import pandas as pd
import numpy as np
import sklearn.cross_validation as cross_val
import time, datetime

X = features = pd.read_csv('./features.csv', index_col='match_id')
y = target = features[u'radiant_win']

del features[u'duration'],features[u'radiant_win'], features[u'tower_status_radiant']
del features[u'tower_status_dire'],features[u'barracks_status_radiant'], features[u'barracks_status_dire']

str_num,col_num = features.shape
col_names = features.columns

### Подход 1: градиентный бустинг "в лоб"
Выведем столбцы с пропусками и сразу же заполним пропущенные элементы

In [4]:
for name in col_names:
    num = features[name].count()
    if num < str_num:
        features[name] = features[name].fillna(value = 0)
        print('%d Null elements in column %s') %(str_num - num, name)

19553 Null elements in column first_blood_time
19553 Null elements in column first_blood_team
19553 Null elements in column first_blood_player1
43987 Null elements in column first_blood_player2
15691 Null elements in column radiant_bottle_time
692 Null elements in column radiant_courier_time
27479 Null elements in column radiant_flying_courier_time
1836 Null elements in column radiant_first_ward_time
16143 Null elements in column dire_bottle_time
676 Null elements in column dire_courier_time
26098 Null elements in column dire_flying_courier_time
1826 Null elements in column dire_first_ward_time


#### Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?
Пропущенные значения в столбцах признаков события first_blood_time, first_blood_team, first_blood_player1 скорее всего связаны с тем, что за первые 5 минут события не произошло. В столбце first_blood_player2 нулевых значений гораздо больше, наверно оно используется, если событие происходит одновременно, а это бывает редко. 
Аналогично столбцы признаков команд.
#### Как называется столбец, содержащий целевую переменную?
Целевой столбец radiant_win. 

In [4]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import grid_search

cv = cross_val.KFold(len(features),n_folds=5, shuffle=True)
for n_estimator in [10,20,30]:
    GBC = GradientBoostingClassifier(n_estimators=n_estimator)
    start_time = datetime.datetime.now()
    score = cross_val.cross_val_score(GBC, features, target, cv = cv, scoring='roc_auc').mean()
    print 'Time elapsed:', datetime.datetime.now() - start_time
    print ('trees number = %d, score = %f') % (n_estimator, score)

Time elapsed: 0:01:49.062409
trees number = 10, score = 0.664641
Time elapsed: 0:03:38.642305
trees number = 20, score = 0.681212
Time elapsed: 0:05:05.655119
trees number = 30, score = 0.688616


#### Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Инструкцию по измерению времени можно найти ниже по тексту. Какое качество при этом получилось? 
Обучение по 30 деревьям шло 3 минуты 48 секунд, при этом качество оказалось 69%, что на 1% процент больше, чем при использовании 20 деревьев, а время время обучение увеличилось примерно на 50%.
#### Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев?
В данном случае увеличение времени обучения не критичное, поэтому возможно имеет смысл использовать 30 деревьев. 
Для ускорения обучения можно, например, сократить количество признаков при помощи метода главных компонент.  

### Подход 2: логистическая регрессия

#### 1. Какое качество получилось у логистической регрессии над всеми исходными признаками? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

X = StandardScaler().fit_transform(features)

for reg in np.arange(0.005,0.01,0.001):
    LR = LogisticRegression(penalty = 'l2', C = reg)
    start_time = datetime.datetime.now()
    predict = cross_val.cross_val_predict(LR, X, target, cv = cv)
    score = roc_auc_score(target,predict)
    print 'Time elapsed:', datetime.datetime.now() - start_time
    print ('regularization strength = %f, score = %f') % (reg, score)

Time elapsed: 0:00:23.111543
regularization strength = 0.005000, score = 0.653996
Time elapsed: 0:00:23.544226
regularization strength = 0.006000, score = 0.653948
Time elapsed: 0:00:20.906021
regularization strength = 0.007000, score = 0.653925
Time elapsed: 0:00:24.493259
regularization strength = 0.008000, score = 0.653905
Time elapsed: 0:00:27.264751
regularization strength = 0.009000, score = 0.653884


Максимальное качество логистической регрессии над всеми признаками получилось 65%, это на 4% меньше, чем показатель градиентного бустинга. Скорость работы логистической регрессии в разы быстрее, чем у градиентного бустинга.

Объяснить разницу

#### Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? Чем вы можете объяснить это изменение?

In [7]:
X = features
del X[u'lobby_type'],X[u'r1_hero'],X[u'r2_hero'],X[u'r3_hero'],X[u'r4_hero'],X[u'r5_hero']
del X[u'd1_hero'],X[u'd2_hero'],X[u'd3_hero'],X[u'd4_hero'],X[u'd5_hero']
X = StandardScaler().fit_transform(X)

In [13]:
for reg in np.arange(0.01,1.0,0.1):#0.31-1.0 best score = 0.654247
    LR = LogisticRegression(penalty = 'l2', C = reg)
    start_time = datetime.datetime.now()
    predict = cross_val.cross_val_predict(LR, X, target, cv = cv)
    score = roc_auc_score(target,predict)
    print 'Time elapsed:', datetime.datetime.now() - start_time
    print ('regularization strength = %f, score = %f') % (reg, score)

Time elapsed: 0:00:16.874939
regularization strength = 0.010000, score = 0.653935
Time elapsed: 0:00:18.756939
regularization strength = 0.110000, score = 0.654194
Time elapsed: 0:00:19.481813
regularization strength = 0.210000, score = 0.654246
Time elapsed: 0:00:15.631732
regularization strength = 0.310000, score = 0.654247
Time elapsed: 0:00:24.950345
regularization strength = 0.410000, score = 0.654247
Time elapsed: 0:00:15.203889
regularization strength = 0.510000, score = 0.654257
Time elapsed: 0:00:16.006301
regularization strength = 0.610000, score = 0.654257
Time elapsed: 0:00:15.067789
regularization strength = 0.710000, score = 0.654247
Time elapsed: 0:00:15.760332
regularization strength = 0.810000, score = 0.654247
Time elapsed: 0:00:17.206616
regularization strength = 0.910000, score = 0.654247


Качество предсказаний не улучшилось. Значит для логистической регрессии эти признаки в числовом виде незначимые.
#### 3 . Сколько различных идентификаторов героев существует в данной игре?
Посчитаем количество уникальных идентификаторов героев

In [73]:
id_num = features.r1_hero.unique().max()
#Проверим выбрав уникальные идентификаторы по всем столбцам с героями
h = np.concatenate([features.r1_hero.unique(),features.r2_hero.unique(),features.r3_hero.unique(),
                    features.r4_hero.unique(),features.r5_hero.unique(),features.d1_hero.unique(),
                    features.d2_hero.unique(),features.d3_hero.unique(),features.d4_hero.unique(),
                    features.d5_hero.unique()])
print id_num, np.max(np.unique(h))

112 112


В игре 108 идентификаторов героев.
#### Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить?
Закодируем информацию о героях с помощью мешка слов.

In [99]:
X_pick = np.zeros((features.shape[0], id_num))

for i, match_id in enumerate(features.index):
    for p in xrange(5):
        X_pick[i, features.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, features.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

#Добавим столбцы к признакам
for i in xrange(id_num):
    features['hero_%d' % i] = X_pick[:,i]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

X = StandardScaler().fit_transform(features)
cv = cross_val.KFold(len(features),n_folds=5, shuffle=True)

for reg in np.linspace(0.001, 1.0, num=20):#np.arange(0.005,0.01,0.001):
    LR = LogisticRegression(penalty = 'l2', C = reg)
    start_time = datetime.datetime.now()
    predict = cross_val.cross_val_predict(LR, X, target, cv = cv)
    score = roc_auc_score(target,predict)
    print 'Time elapsed:', datetime.datetime.now() - start_time
    print ('regularization strength = %f, score = %f') % (reg, score) #0.6819 1.8
    print predict

Time elapsed: 0:00:21.485700
regularization strength = 0.001000, score = 0.682015
Time elapsed: 0:00:29.415787
regularization strength = 0.053579, score = 0.682058
Time elapsed: 0:00:32.174494
regularization strength = 0.106158, score = 0.682018
Time elapsed: 0:00:28.528078
regularization strength = 0.158737, score = 0.682027
Time elapsed: 0:00:29.536610
regularization strength = 0.211316, score = 0.682038
Time elapsed: 0:00:31.432241
regularization strength = 0.263895, score = 0.682049
Time elapsed: 0:00:31.598233
regularization strength = 0.316474, score = 0.682050


In [115]:
np.linspace(0.001, 1.0, num=20)

array([ 0.001     ,  0.05357895,  0.10615789,  0.15873684,  0.21131579,
        0.26389474,  0.31647368,  0.36905263,  0.42163158,  0.47421053,
        0.52678947,  0.57936842,  0.63194737,  0.68452632,  0.73710526,
        0.78968421,  0.84226316,  0.89484211,  0.94742105,  1.        ])